In [8]:
import sys
import os

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from PyQt5.QtWidgets import QMainWindow, QWidget, QGridLayout, QVBoxLayout, QHBoxLayout, QApplication
from pypylon import pylon

from apps.manager import initManagers, WidgetManager, ConfigManager, DataManager, ControlManager, LayoutManager
from apps.control import ViewControl
from apps.layout.basler_camera_gui import makeLayoutBaslerCameraGUI
from apps.bind_func.utils import *
from apps.bind_func.basler_camera_gui import *

class BaslerCameraGUI(QMainWindow):
    def __init__(self):
        QMainWindow.__init__(self)
        self.app_name = "BaslerCameraGUI"
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager, self.layout_manager = initManagers(
            WidgetManager(), ConfigManager(), DataManager(), ControlManager(), LayoutManager()
        )

        # set window title and geometry
        self.setWindowTitle(self.app_name)
        # self.setGeometry(0, 0, 1920, 900) # hardcoded !!!

        self.initUI()

        # setup managers
        self.setupManagers()

        # setup controls
        self.setupControls()

        # bind functions
        self.bindFuncAllWidgets()

    """
    setup UI function
    """
    def initUI(self):
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout_main = QGridLayout(self.central_widget)
        self.layout_main.addLayout(
            makeLayoutBaslerCameraGUI(self.widget_manager, self.config_manager),
            0, 0
        )

    def setupManagers(self):
        # ConfigManager
        self.config_manager.current_app = self.app_name

    def setupControls(self):
        # view control
        self.control_manager.view_control = ViewControl(
            self.widget_manager.dict_view["view_camera"],
            self.widget_manager.dict_scene["view_camera"],
            self.data_manager,
            self.widget_manager,
            self.config_manager,
            self.control_manager
        )
        # add camera view layer
        self.control_manager.view_control.addViewLayer("camera")

        # store flags
        self.control_manager.dict_flag = {}
        self.control_manager.dict_flag["is_capture"] = False

        self.control_manager.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        self.control_manager.converter = pylon.ImageFormatConverter()
    
    """
    bindFunc
    """
    def bindFuncAllWidgets(self):
        bindFuncExit(q_button=self.widget_manager.dict_button["exit"], q_window=self)
        bindFuncButtonCaptureSingle(
            self.widget_manager.dict_button["capture_single"],
            self,
            self.widget_manager,
            self.control_manager,
            self.data_manager
            )


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    gui = BaslerCameraGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

In [3]:
gui.data_manager.im.shape

(1024, 1280, 3)

In [4]:
import numpy as np
import cv2

img = np.random.randint(0, 255, (480, 640), dtype=np.uint8)

cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

array([[[ 85,  85,  85],
        [145, 145, 145],
        [191, 191, 191],
        ...,
        [183, 183, 183],
        [ 93,  93,  93],
        [240, 240, 240]],

       [[192, 192, 192],
        [180, 180, 180],
        [187, 187, 187],
        ...,
        [249, 249, 249],
        [  2,   2,   2],
        [  3,   3,   3]],

       [[ 61,  61,  61],
        [103, 103, 103],
        [ 23,  23,  23],
        ...,
        [ 77,  77,  77],
        [  4,   4,   4],
        [ 91,  91,  91]],

       ...,

       [[208, 208, 208],
        [ 63,  63,  63],
        [233, 233, 233],
        ...,
        [ 89,  89,  89],
        [ 46,  46,  46],
        [232, 232, 232]],

       [[ 84,  84,  84],
        [123, 123, 123],
        [115, 115, 115],
        ...,
        [ 20,  20,  20],
        [ 11,  11,  11],
        [123, 123, 123]],

       [[130, 130, 130],
        [127, 127, 127],
        [181, 181, 181],
        ...,
        [239, 239, 239],
        [150, 150, 150],
        [  8,   8,   8]]

In [7]:
gui.widget_manager.dict_scene

{'view_camera': <PyQt5.QtWidgets.QGraphicsScene at 0x2340409d280>}